# 제주도 핫플레이스 - 인스타그램 크롤링

In [2]:
import requests
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

In [ ]:
chromedirver = 'D:\download\chromedriver_win32\chromedriver'
driver = webdriver.Chrome(chromedirver)

In [3]:
!apt-get update > /dev/null
!pip install selenium > /dev/null
!apt install chromium-chromedriver > /dev/null

������ ��θ� ã�� �� �����ϴ�.
������ ��θ� ã�� �� �����ϴ�.
������ ��θ� ã�� �� �����ϴ�.


In [ ]:
insta_url = 'https://www.instagram.com'
driver.get(insta_url)
time.sleep(1)

In [ ]:
with open('password.txt') as f:
    password = f.read()

In [ ]:
len(password)

In [ ]:
with open('id.txt') as f:
    id = f.read()

In [ ]:
len(id)

In [ ]:
email = id
input_email = driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP')      #공백은 .으로 변경해줘야 함
input_email.clear()
input_email.send_keys(email)

In [ ]:
input_pwd = driver.find_element_by_name('password')      #공백은 .으로 변경해줘야 함
input_pwd.clear()
input_pwd.send_keys(password)
input_pwd.submit()
time.sleep(1)

In [ ]:
# 브라우저 저장 나중에 하기
try:
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()
    time.sleep(1)
except:
    pass
time.sleep(10)


In [ ]:
# 알림설정 나중에 하기
try:
    driver.find_element_by_css_selector('.aOOlW.HoLwm').click()
    time.sleep(1)
except:
    pass

### 2.제주도맛집으로 검색

In [ ]:
from urllib.parse import quote

In [ ]:
keyword = '제주도맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'

In [ ]:
driver.get(url)
time.sleep(20)

### 3. 첫번째 게시글 열기

In [ ]:
driver.find_element_by_css_selector('._9AhH0').click()
time.sleep(1)


### 4. 게시글 정보 가져오기

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# 1) 게시글 가져오기
import unicodedata
try:
    content = soup.select_one('div.C4VMK > span').text
    content = unicodedata.normalize('NFC', content)
except:
    content = ' '
content

In [ ]:
# 2) 본문 내용에서 해시태그 가져오기, '#[^\s#,\\]+'

import re
tags = re.findall(r'#[^\s#,\\]+', content)
tags

In [ ]:
# 3) 작성일자 정보 가져오기
date = soup.select_one('time.FH9sR.Nzb55')['title']
date

In [ ]:
date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]
date

In [ ]:
# 4) 좋아요 수 가져오기
try:
    like = soup.select_one('div.Nm9Fw').text[4:-1]
except:
    like = 0
like

In [ ]:
# 5) 위치정보 가져오기
try:
    place = soup.select('div.M30cS')[0]
    place = unicodedata.normalize('NFC', place)
except:
    place = ' '
place

In [ ]:
row = [content, date, like, place, tags]
row

### 5. 다음 게시글로 이동

In [ ]:
driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
time.sleep(2)

In [ ]:
# 6을 하기위해 popup 화면을 닫기 (x 클릭) 셀렉터 버전
driver.find_element_by_css_selector('body > div._2dDPU.CkGkG > div.Igw0E.IwRSH.eGOV_._4EzTm.BI4qX.qJPeX.fm1AK.TxciK.yiMZG > button > div > svg').click()


In [ ]:
# xpath 버전
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()

### 6. 여러개의 게시글 가져오기

### - 제주도맛집, 제주맛집, 제주도관광, 제주여행

In [ ]:
keyword = '제주도맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
driver.get(url)
time.sleep(20)
driver.find_element_by_css_selector('._9AhH0').click()
time.sleep(1)

In [ ]:
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    try:
        content = soup.select_one('div.C4VMK > span').text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '
    tags = re.findall(r'#[^\s#,\\]+', content)
    date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]
    try:
        like = soup.select_one('div.Nm9Fw').text[4:-1]
    except:
        like = 0
    try:
        place = soup.select('div.M30cS')[0]
        place = unicodedata.normalize('NFC', place)
    except:
        place = ' '
    return [content, tags, date, like, place]

In [ ]:
# 인스타 피드 크롤링 200개가 한계
results = []
from tqdm import notebook

for _ in notebook.tqdm(range(200)):
    try:
        row = get_content(driver)
        results.append(row)
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(1)
    except:
        time.sleep(3)
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(1)


In [ ]:
df = pd.DataFrame(results, columns=['content', 'tags', 'date', 'like', 'place'])
df

In [ ]:
df.to_csv('data/제주도맛집.csv', index=False)

In [ ]:
for keyword in ['제주도맛집', '제주맛집', '제주도관광', '제주여행']:
    url = f'https://www.instagram.com/explore/tags/{quote(keyword)}'
    driver.get(url)
    time.sleep(20)
    driver.find_element_by_css_selector('._9AhH0').click()
    time.sleep(1)

    results = []
    from tqdm import notebook

    for _ in notebook.tqdm(range(200), desc=f'{keyword}'):
        try:
            row = get_content(driver)
            results.append(row)
            driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
            time.sleep(2)
        except:
            driver.refresh()
            time.sleep(20)
            driver.find_element_by_css_selector('._9AhH0').click()
            time.sleep(2)
            driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
            time.sleep(2)

    df = pd.DataFrame(results, columns=['content', 'tags', 'date', 'like', 'place'])
    df.to_csv(f'data/{keyword}.csv', index=False)
    driver.refresh()

In [ ]:
# xpath 버전
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()

In [ ]:
df

In [ ]:
driver.close()

### 4가지 csv파일의 중복을 제거한 후 통합 저장

In [5]:
jeju_df = pd.DataFrame([])

for keyword in ['제주도맛집', '제주맛집', '제주도관광', '제주여행']:
    df = pd.read_csv(f'data/{keyword}.csv')
    jeju_df = jeju_df.append(df)

jeju_df


,content,date,like,place,tags
0,.#연동 #모니카옛날통닭간판부터 맛집포스 작렬하는 찐맛집이젠 인생치킨이라고 말하기도...,"['#연동', '#모니카옛날통닭간판부터', '#제주맛집', '#제주도맛집', '#제...",2021-08-19,"1,113",NaN
1,<광고>섭지코지에서 한끼 맛있게 드시고 싶다면...특히 점심메뉴로 강력추천드리고요 ...,"['#섭지코지한끼🍱오픈', '#섭지코지맛집', '#섭지코지', '#섭지코지카페', ...",2021-08-18,0,NaN
2,.< 제주해물라면 맛집모음! > 일단저장@ 해물 아낌없이 주는 해물라면 맛집!바다뷰...,"['#제주맛집', '#제주도맛집', '#제주공항근처맛집', '#제주시맛집', '#애...",2021-08-18,"1,008",NaN
3,"협찬제주애월맛집 ""랍스터빈""주소 : 제주시 애월읍 애월해안로 752전화 : 064-...","['#애월맛집', '#애월카페', '#애월읍맛집', '#제주애월맛집', '#제주도맛...",2021-08-18,0,NaN
4,.한림 #문도지오름방목된 조랑말과 교감까지 할 수이따규~일몰 시간에 맞춰서 가면 멋...,"['#문도지오름방목된', '#제주맛집', '#제주도맛집', '#제주공항근처맛집', ...",2021-08-17,"1,304",NaN
...,...,...,...,...,...
109,"ო̤̜어깨 깡패 😏8, 9월 예약 받고 있습니다 🤗문의는 프로필의 오픈 카톡으로 부...",[],2021-08-19,19,제주시청 근처
110,.막 찍어도 화보 같은 월정리 카페 콧수염고객님의 멋진 사진 감사합니다 🤳제주시 구...,"['#구좌읍카페', '#함덕맛집', '#제주', '#제주도맛집', '#월정리', '...",2021-08-19,3,NaN
111,사장님 알로뷰 …❤️,[],2021-08-19,43,One and only - 원앤온리 제주
112,#제주여행 #1일차#사진못찍음,"['#제주여행', '#1일차', '#사진못찍음']",2021-08-19,4,NaN


In [6]:
jeju_df.to_csv(f'data/jeju_df.csv', index=False)

In [ ]:
jeju_df = []